# Assignment 6

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

朴素贝叶斯中，独立假设是指用于分类的特征在类别确定的前提下都是条件独立的。也就是说：$$P(a_1, a_2, \cdots, a_n|v_j) = P(a_1|v_j)P(a_2|v_j)\cdots P(a_n|v_j)$$

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

在给定数据D和假设空间H前提下，通过MAP或ML来对输入进行预测。

MAP, 即最大化后验概率。$$h_{MAP} = {\arg\max}_h P(h|D) = {\arg\max}_h {\frac{P(D|h)P(h)}{P(D)}}={\arg\max}_h{P(D|h)P(h)}$$
ML，即最大化似然估计概率。如果对于假设空间H中的所有假设h，概率相同，我们就得到了ML模型。$$h_{ML} = {\arg\max}_h{P(D|h)}$$

#### 3. What is support vector in SVM?
支持向量(support vector)指对分割超平面有贡献的样本点。因为分割超平面是由落在分割边界上的点来决定的，其他点不管有多少，都不影响分割超平面的位置，因此把那些支撑起分割超平面的样本点成为SVM中的支持向量。

#### 4. What is the intuition behind SVM ?
SVM利用直线（或高维空间中的超平面）对数据进行分类，使得离直线（或超平面）最近的点间隔最大化。

#### 5. Shortly describ what 'random' means in random forest ?
随机森林是“随机”从样本中采样，然后生成决策树。这样重复多次，生成许多决策树。分类时，让这些决策数各自决策，然后少数服从多数，获得答案。

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?
遍历每个特征，在每个特征中选择该特征下的每个值作为其分裂点，计算增益损失。当遍历完所有特征之后，增益损失最大的特征值将作为其分裂点。

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
import ast
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
data = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')
data[:3]

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [3]:
# 查看source是新华社和非新华社的比例
data['publisher'] = np.where(data['source'] == '新华社', 1, 0)
data['publisher'].value_counts()

1    78661
0    10950
Name: publisher, dtype: int64

In [4]:
# 去掉content中的NAN
data.dropna(subset=['content'], inplace=True)
# 分词处理
stopword = [word.strip() for word in open('百度停用词表.txt') if word.strip != '']
sentences = []
for content in data['content']:
    sentence = [word for word in jieba.cut(content) if word not in stopword and word not in ['\u3000', '\n','\r\n']]
    sentences.append(sentence)
    
with open('cutwords.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
       f.write(' '.join(sentence) + '\n')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/x7/q7pcyqcj37l3psvcymyg7pqc0000gn/T/jieba.cache
Loading model cost 0.699 seconds.
Prefix dict has been built successfully.


In [5]:
with open('cutwords.txt','r') as f:
    sentences = [line.split() for line in f]

# TF-IDF,朴素贝叶斯

In [9]:
tfidf_model = TfidfVectorizer(sentences, min_df= 50, max_df= 0.3)

X = tfidf_model.fit_transform(data['content'])
Y = data['publisher'].values
# 分层抽样划分数据集
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size= 0.2, stratify= Y)
print(f'训练集X数量为：{train_x.shape[0]}')
print(f'测试集X数量为：{test_x.shape[0]}')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



训练集X数量为：69643
测试集X数量为：17411


In [12]:
from sklearn.naive_bayes import MultinomialNB

bayes_model = MultinomialNB()
bayes_model.fit(train_x, train_y)
print(f'Accuracy with Navie Bayes: {bayes_model.score(test_x, test_y)}','\n')

y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names= ['非新华社','新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns= ['非新华社','新华社'], index= ['非新华社','新华社'])

Accuracy with Navie Bayes: 0.9503187639997702 

              precision    recall  f1-score   support

        非新华社       0.73      0.78      0.75      1679
         新华社       0.98      0.97      0.97     15732

    accuracy                           0.95     17411
   macro avg       0.85      0.87      0.86     17411
weighted avg       0.95      0.95      0.95     17411



,非新华社,新华社
非新华社,1305,374
新华社,491,15241


In [14]:
# 数据不平衡，新华社非新华社分类效果不好。 试用SMOTE采样
X_smote1, Y_smote1 = SMOTE().fit_sample(train_x, train_y)
bayes_model.fit(X_smote1, Y_smote1)
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')

y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8656022055022686
              precision    recall  f1-score   support

        非新华社       0.42      0.97      0.58      1679
         新华社       1.00      0.85      0.92     15732

    accuracy                           0.87     17411
   macro avg       0.71      0.91      0.75     17411
weighted avg       0.94      0.87      0.89     17411



,非新华社,新华社
非新华社,1632,47
新华社,2293,13439


In [15]:
X_smote2, Y_smote2 = BorderlineSMOTE(kind='borderline-1').fit_sample(train_x, train_y)
bayes_model.fit(X_smote2, Y_smote2)
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')

y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8556085233473092
              precision    recall  f1-score   support

        非新华社       0.40      0.98      0.57      1679
         新华社       1.00      0.84      0.91     15732

    accuracy                           0.86     17411
   macro avg       0.70      0.91      0.74     17411
weighted avg       0.94      0.86      0.88     17411



,非新华社,新华社
非新华社,1645,34
新华社,2480,13252


In [16]:
X_smote3, Y_smote3 = BorderlineSMOTE(kind='borderline-2').fit_sample(train_x, train_y)
bayes_model.fit(X_smote3, Y_smote3)
print(f'Accuracy with Naive Bayes: {bayes_model.score(test_x, test_y)}')

y_pred_bayes = bayes_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with Naive Bayes: 0.8605479294698754
              precision    recall  f1-score   support

        非新华社       0.40      0.92      0.56      1679
         新华社       0.99      0.85      0.92     15732

    accuracy                           0.86     17411
   macro avg       0.70      0.89      0.74     17411
weighted avg       0.93      0.86      0.88     17411



,非新华社,新华社
非新华社,1540,139
新华社,2289,13443


### 使用SMOTE总体上Accuracy会降低，但对于非新华社的recall会提升

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).

# 逻辑回归模型

In [19]:
from sklearn.linear_model import LogisticRegressionCV

C = [1, 10, 100, 1000]
lr_model = LogisticRegressionCV(Cs= C, class_weight= 'balanced')
lr_model.fit(train_x, train_y)
print(f'Accuracy with LogisticRegression: {lr_model.score(test_x, test_y)}')
y_pred_bayes = lr_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with LogisticRegression: 0.9083912469128712
              precision    recall  f1-score   support

        非新华社       0.51      0.92      0.66      1679
         新华社       0.99      0.91      0.95     15732

    accuracy                           0.91     17411
   macro avg       0.75      0.91      0.80     17411
weighted avg       0.94      0.91      0.92     17411



,非新华社,新华社
非新华社,1538,141
新华社,1454,14278


In [20]:
lr_model.fit(X_smote2, Y_smote2)
print(f'Accuracy with LogisticRegression: {lr_model.score(test_x, test_y)}')
y_pred_lr = lr_model.predict(test_x)
print(classification_report(test_y, y_pred_lr, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_lr), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with LogisticRegression: 0.9104589052897594
              precision    recall  f1-score   support

        非新华社       0.52      0.91      0.66      1679
         新华社       0.99      0.91      0.95     15732

    accuracy                           0.91     17411
   macro avg       0.75      0.91      0.81     17411
weighted avg       0.94      0.91      0.92     17411



,非新华社,新华社
非新华社,1525,154
新华社,1405,14327


### 支持向量机 SVM

In [36]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

svc_param = {'C': [0.1, 1, 10, 100],
         'kernel': ['linear', 'rbf'],
         'class_weight': ['balanced']}

cv = StratifiedKFold(shuffle=True)
clf = SVC()
cv_svc = GridSearchCV(clf, svc_param, cv=cv)
cv_svc.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'class_weight': ['balanced'],
                         'kernel': ['linear', 'rbf']})

In [37]:
cv_svc.best_params_

{'C': 100, 'class_weight': 'balanced', 'kernel': 'rbf'}

In [38]:
svc_model = SVC(C=10, class_weight='balanced', kernel='rbf')
svc_model.fit(train_x, train_y)
print(f'Accuracy with SVM: {svc_model.score(test_x, test_y)}')
y_pred_bayes = svc_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with SVM: 0.9256217333869393
              precision    recall  f1-score   support

        非新华社       0.58      0.83      0.68      1679
         新华社       0.98      0.94      0.96     15732

    accuracy                           0.93     17411
   macro avg       0.78      0.88      0.82     17411
weighted avg       0.94      0.93      0.93     17411



,非新华社,新华社
非新华社,1393,286
新华社,1009,14723


In [39]:
svc_model.fit(X_smote2, Y_smote2)
print(f'Accuracy with SVM: {svc_model.score(test_x, test_y)}')
y_pred_svm = svc_model.predict(test_x)
print(classification_report(test_y, y_pred_svm, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_svm), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with SVM: 0.9275745218540004
              precision    recall  f1-score   support

        非新华社       0.59      0.82      0.69      1679
         新华社       0.98      0.94      0.96     15732

    accuracy                           0.93     17411
   macro avg       0.78      0.88      0.82     17411
weighted avg       0.94      0.93      0.93     17411



,非新华社,新华社
非新华社,1375,304
新华社,957,14775


### 随机森林模型

In [26]:
from sklearn.ensemble import RandomForestClassifier

# 先对n_estimators调参
rf_param = {'n_estimators': [200, 400, 600],
         'min_samples_split': [18],
         'max_depth' : [14]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [14], 'min_samples_split': [18],
                         'n_estimators': [200, 400, 600]})

In [27]:
print(cv_rf.best_params_)
print(cv_rf.best_score_)

{'max_depth': 14, 'min_samples_split': 18, 'n_estimators': 200}
0.9132145220433371


In [28]:
# 固定n_estimators，对min_samples_split与max_depth调参
# 经过多次调参，最后确定的参数如下：
rf_param = {'n_estimators': [200],
         'min_samples_split': [36, 40, 44],
         'max_depth' : [260, 270, 280]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [260, 270, 280],
                         'min_samples_split': [36, 40, 44],
                         'n_estimators': [200]})

In [29]:
print(cv_rf.best_params_)
print(cv_rf.best_score_)

{'max_depth': 270, 'min_samples_split': 36, 'n_estimators': 200}
0.959507791813154


In [30]:
# 对max_features调参
rf_param = {'n_estimators': [200],
         'min_samples_split': [36],
         'max_depth': [280],
         'max_features': ["auto", "sqrt", "log2"]}

cv = StratifiedKFold(shuffle=True)
clf = RandomForestClassifier()
cv_rf = GridSearchCV(clf, rf_param, cv=cv)
cv_rf.fit(train_x, train_y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [280],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [36], 'n_estimators': [200]})

In [31]:
print(cv_rf.best_params_)
print(cv_rf.best_score_)

{'max_depth': 280, 'max_features': 'sqrt', 'min_samples_split': 36, 'n_estimators': 200}
0.9593929070581094


In [32]:
rf_model = RandomForestClassifier(n_estimators=200, min_samples_split=36, max_features='auto', max_depth=280)
rf_model.fit(train_x, train_y)

RandomForestClassifier(max_depth=280, min_samples_split=36, n_estimators=200)

In [33]:
print(f'Accuracy with RandomForest: {rf_model.score(test_x, test_y)}')
y_pred_bayes = rf_model.predict(test_x)
print(classification_report(test_y, y_pred_bayes, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_bayes), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with RandomForest: 0.9611165355235196
              precision    recall  f1-score   support

        非新华社       0.92      0.66      0.76      1679
         新华社       0.96      0.99      0.98     15732

    accuracy                           0.96     17411
   macro avg       0.94      0.82      0.87     17411
weighted avg       0.96      0.96      0.96     17411



,非新华社,新华社
非新华社,1100,579
新华社,98,15634


In [34]:
rf_model.fit(X_smote2, Y_smote2)

RandomForestClassifier(max_depth=280, min_samples_split=36, n_estimators=200)

In [35]:
print(f'Accuracy with RandomForest: {rf_model.score(test_x, test_y)}')
y_pred_rf = rf_model.predict(test_x)
print(classification_report(test_y, y_pred_rf, target_names=['非新华社', '新华社']))
pd.DataFrame(confusion_matrix(test_y, y_pred_rf), columns=['非新华社', '新华社'], index=['非新华社', '新华社'])

Accuracy with RandomForest: 0.8996036988110965
              precision    recall  f1-score   support

        非新华社       0.49      0.91      0.64      1679
         新华社       0.99      0.90      0.94     15732

    accuracy                           0.90     17411
   macro avg       0.74      0.90      0.79     17411
weighted avg       0.94      0.90      0.91     17411



,非新华社,新华社
非新华社,1526,153
新华社,1595,14137


## 总结

In [41]:
import cufflinks as cf
import matplotlib.pyplot as plt
%matplotlib inline
cf.set_config_file(offline=True)

model_name = ['Bayes', 'LR', 'SVM', 'RF']
result = {
    'Accuracy': [0.951, 0.909, 0.928, 0.959],
    'Accuracy_with_smote': [0.867, 0.913, 0.927, 0.900],
    'Recall': [0.77, 0.94, 0.85, 0.64],
    'Recall_with_smote': [0.98, 0.92, 0.86, 0.92]
        }

In [42]:
result = pd.DataFrame(result, index=model_name)
result

,Accuracy,Accuracy_with_smote,Recall,Recall_with_smote
Bayes,0.951,0.867,0.77,0.98
LR,0.909,0.913,0.94,0.92
SVM,0.928,0.927,0.85,0.86
RF,0.959,0.900,0.64,0.92


## RandomForest的准确率最高